In [ ]:
# 환경변수 가져오기
# openai_api_key = os.getenv("OPENAI_API_KEY")
# serpapi_key = os.getenv("SERPAPI_API_KEY")

# 또는 다음과 같이 직접 키 입력 (개발)
# os.environ["OPENAI_API_KEY"] = ""  # 자신의 OpenAI 키
# os.environ["SERPAPI_API_KEY"] = ""

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

#### 다음 실습 코드는 학습 목적으로만 사용 바랍니다. 문의 : audit@korea.ac.kr 임성열 Ph.D.


LangChain Expression Language(LCEL)는 랭체인에서 체인을 간결하게 구성하는 문법입니다.   
먼저, LCEL에서 체인이 구성되는 기본적인 구조에 대해 알아봅시다.


In [1]:
!pip install openai langchain langchain_openai -q

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#os.environ["OPENAI_API_KEY"] = "<OpenAI_API의 API 키>"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


기존에는 `LLM.invoke(prompt.format(___))` 형태로 실행을 했었는데요.    
LCEL의 가장 큰 특징은, Chain의 구성 요소를 **|**  (파이프)로 연결하여 한 번에 실행한다는 점입니다. 예시를 보겠습니다.

In [4]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


# topic에 대한 영어 농담을 하고, 이것이 왜 농담인지 한국어로 설명하세요.
fun_chat_template = ChatPromptTemplate.from_messages([
    ('user',"""tell me an English joke about {topic},
also, explain in Korean why it is fun for english-speakers.
Include translation of the joke.""")
])


llm=ChatOpenAI(temperature=0.5, model='gpt-4o-mini',max_tokens=1000)
# LLM 유머는 지능에 비례



In [5]:
response = llm.invoke(fun_chat_template.format_messages(topic='AI'))
response

AIMessage(content='**Joke:**\nWhy did the AI break up with its partner?  \nBecause it had too many "issues" to resolve!\n\n**Korean Explanation:**\n이 농담은 영어 사용자에게 재미있는 이유는 \'issues\'라는 단어의 이중적 의미 때문입니다. \'Issues\'는 문제를 의미할 뿐만 아니라 감정적 문제나 갈등을 의미하기도 합니다. AI는 감정을 느끼지 않지만, 인간의 관계에서 자주 발생하는 문제를 언급함으로써 유머를 만듭니다. 이처럼 AI가 인간의 감정을 이해하지 못한다는 점에서 아이러니가 발생하고, 그로 인해 웃음을 유발합니다.\n\n**Translation of the joke in Korean:**\n왜 AI는 파트너와 헤어졌나요?  \n너무 많은 "문제"를 해결해야 했기 때문입니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 183, 'prompt_tokens': 35, 'total_tokens': 218, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-C9le1FjBb3TiGQ9tYTt1wpUgOt0Kt', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': Non

In [6]:
print(response.content)

**Joke:**
Why did the AI break up with its partner?  
Because it had too many "issues" to resolve!

**Korean Explanation:**
이 농담은 영어 사용자에게 재미있는 이유는 'issues'라는 단어의 이중적 의미 때문입니다. 'Issues'는 문제를 의미할 뿐만 아니라 감정적 문제나 갈등을 의미하기도 합니다. AI는 감정을 느끼지 않지만, 인간의 관계에서 자주 발생하는 문제를 언급함으로써 유머를 만듭니다. 이처럼 AI가 인간의 감정을 이해하지 못한다는 점에서 아이러니가 발생하고, 그로 인해 웃음을 유발합니다.

**Translation of the joke in Korean:**
왜 AI는 파트너와 헤어졌나요?  
너무 많은 "문제"를 해결해야 했기 때문입니다!


-----------
LCEL의 인터페이스는 조금 다릅니다.

템플릿과 llm 모델을 설정하고, 이를 하나로 묶어 체인을 생성합니다.

In [7]:
joke = fun_chat_template | llm
# shift + \ = |||||
joke

ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='tell me an English joke about {topic},\nalso, explain in Korean why it is fun for english-speakers.\nInclude translation of the joke.'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x111f65890>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x111f65fd0>, root_client=<openai.OpenAI object at 0x111f64c10>, root_async_client=<openai.AsyncOpenAI object at 0x111f65a10>, model_name='gpt-4o-mini', temperature=0.5, model_kwargs={}, openai_api_key=SecretStr('**********'), max_tokens=1000)

이후, 체인의 invoke를 실행하며 입력 포맷을 전달하면, 순서대로 체인이 실행되며 최종 결과로 연결됩니다.    
입력 포맷은 Dict 형식으로 전달합니다.

In [8]:
response = joke.invoke({'topic':'apple'})
# invoke에 대한 입력은 딕셔너리로 처리 ':'
# 단, 매개 변수가 1개일 때는
# joke.invoke('apple')과 같이 문자열로 입력할 수도 있음
response

AIMessage(content="**Joke:**\nWhy did the apple stop in the middle of the road?  \nBecause it ran out of juice!\n\n**Korean Explanation:**\n이 농담은 영어 사용자에게 재미있는 이유는 'juice'라는 단어의 이중적 의미 때문입니다. 'Juice'는 사과의 주스(즙)를 의미할 뿐만 아니라, 차나 다른 기계의 에너지를 의미하기도 합니다. 따라서 사과가 도로에서 멈춘 이유가 '주스가 다 떨어졌다'는 것은 사과가 물리적으로 주스가 없어서 멈춘 것과 동시에, 비유적으로는 힘이 다 떨어져서 멈춘 것처럼 해석될 수 있어 웃음을 줍니다.\n\n**Translation of the Joke:**\n왜 사과가 도로 한가운데에서 멈췄나요?  \n주스가 다 떨어졌기 때문이에요!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 192, 'prompt_tokens': 35, 'total_tokens': 227, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C9lfI1ze05xXsKFyxhUYIc5e14rAs', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--7826d1bb-50

In [9]:
print(response.content)

**Joke:**
Why did the apple stop in the middle of the road?  
Because it ran out of juice!

**Korean Explanation:**
이 농담은 영어 사용자에게 재미있는 이유는 'juice'라는 단어의 이중적 의미 때문입니다. 'Juice'는 사과의 주스(즙)를 의미할 뿐만 아니라, 차나 다른 기계의 에너지를 의미하기도 합니다. 따라서 사과가 도로에서 멈춘 이유가 '주스가 다 떨어졌다'는 것은 사과가 물리적으로 주스가 없어서 멈춘 것과 동시에, 비유적으로는 힘이 다 떨어져서 멈춘 것처럼 해석될 수 있어 웃음을 줍니다.

**Translation of the Joke:**
왜 사과가 도로 한가운데에서 멈췄나요?  
주스가 다 떨어졌기 때문이에요!


## 실습) 매개변수가 2개인 Prompt-LLM Chain 생성하기   
임의의 ChatPromptTemplate를 만들고, 2개의 매개변수를 받도록 구성하여 체인을 만들고 실행하세요.

In [10]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


# topic에 대한 영어 농담을 하고, 이것이 왜 농담인지 한국어로 설명하세요.
fun_chat_template1 = ChatPromptTemplate.from_messages([
    ('user',"""tell me an English joke about {topic},
also, explain in Korean why it is fun for english-speakers.
Include translation of the joke.""")
])

fun_chat_template2 = ChatPromptTemplate.from_messages([
    ('user',"""tell me an Korean joke about {topic},
also, explain in Korean why it is fun for korean-speakers.
Include translation of the joke.""")
])


llm=ChatOpenAI(temperature=0.5, model='gpt-4o-mini',max_tokens=1000)
# LLM 유머는 지능에 비례

In [11]:
response1 = llm.invoke(fun_chat_template1.format_messages(topic='AI'))
response2 = llm.invoke(fun_chat_template2.format_messages(topic='AI'))

print(response1.content)
print(response2.content)

**Joke:**
Why did the robot go on a diet?  
Because it had too many bytes!

**Translation:**
로봇은 왜 다이어트를 시작했나요?  
너무 많은 바이트가 있었기 때문이에요!

**Explanation in Korean:**
이 농담은 "바이트(byte)"라는 단어의 이중 의미 때문에 재미있습니다. 영어에서 "바이트"는 컴퓨터 데이터의 단위를 의미하지만, "바이트"는 "한 입"이라는 뜻의 "bite"와 발음이 비슷합니다. 그래서 로봇이 다이어트를 시작한 이유가 너무 많은 데이터(바이트)를 섭취했기 때문이라는 설정이 웃음을 자아냅니다. 영어 사용자들은 이러한 언어 유희와 발음의 유사성에서 재미를 느끼기 때문에 이 농담이 재미있습니다.
### Joke:
왜 AI는 항상 농담을 잘 못할까요?  
왜냐하면, 그들은 항상 "데이터"를 너무 많이 생각해요!

### Translation:
Why does AI always struggle with jokes?  
Because they think too much about "data"!

### Explanation in Korean:
이 농담은 한국어 사용자에게 재미있습니다. "데이터"라는 단어가 두 가지 의미로 해석될 수 있기 때문입니다. 첫 번째는 AI가 데이터를 분석하는 과정에서 농담의 유머를 이해하지 못하는 상황을 나타내고, 두 번째는 "데이터"라는 말이 "딱딱하다"는 느낌을 주어 농담이 재미없다는 것을 암시합니다. 이러한 언어유희가 한국인들에게 친숙하고 웃음을 자아내기 때문에 재미있게 느껴집니다.


In [12]:
joke = fun_chat_template1 | fun_chat_template2 | llm
# shift + \ = |||||
print(joke)


first=ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='tell me an English joke about {topic},\nalso, explain in Korean why it is fun for english-speakers.\nInclude translation of the joke.'), additional_kwargs={})]) middle=[ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='tell me an Korean joke about {topic},\nalso, explain in Korean why it is fun for korean-speakers.\nInclude translation of the joke.'), additional_kwargs={})])] last=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x11226a5d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11226aa90>, root_client=<openai.Op

LCEL의 체인에는 **파서(Parser)** 를 추가할 수 있습니다.    
파서는 출력 형식을 변환합니다.

StrOutputParser : 출력 결과를 String 형식으로 변환합니다.

In [14]:
from langchain_core.output_parsers import StrOutputParser

recipe_template=ChatPromptTemplate.from_messages([
    ('system','당신은 전세계의 조리법을 아는 쉐프입니다.'),
    ('user','저는 {ingredient}를 이용한 환상적인 외국 음식을 만들고 싶습니다. 추천해주세요!')
])

In [15]:
recipe_chain = recipe_template | llm | StrOutputParser()
response = recipe_chain.invoke({'ingredient':'와인'})
print(response)

와인을 이용한 환상적인 외국 음식으로는 "Coq au Vin" (코크 오 뱅)이라는 프랑스 요리를 추천합니다. 이 요리는 닭고기를 레드 와인, 버섯, 양파, 베이컨과 함께 조리하여 깊고 풍부한 맛을 내는 전통적인 프랑스 요리입니다. 아래는 Coq au Vin의 간단한 레시피입니다.

### Coq au Vin 레시피

#### 재료:
- 닭고기 (다리 또는 가슴살) 1kg
- 레드 와인 750ml (부르고뉴 와인 추천)
- 베이컨 150g (또는 판체타)
- 양파 2개 (작게 썰기)
- 마늘 4쪽 (다지기)
- 당근 2개 (슬라이스)
- 버섯 200g (슬라이스)
- 밀가루 2큰술
- 올리브 오일 2큰술
- 타임 1작은술
- 월계수 잎 1장
- 소금과 후추 (맛 조절용)
- 파슬리 (장식용)

#### 조리법:
1. **준비**: 닭고기를 소금과 후추로 간을 해주세요. 베이컨은 작은 조각으로 자릅니다.

2. **베이컨 볶기**: 큰 냄비에 올리브 오일을 두르고 베이컨을 넣어 바삭하게 볶아주세요. 볶은 베이컨은 따로 빼놓습니다.

3. **닭고기 굽기**: 같은 냄비에 닭고기를 넣고 겉면이 노릇해질 때까지 볶습니다. 그런 다음 밀가루를 뿌리고 잘 섞어주세요.

4. **야채 추가**: 볶은 닭고기에 양파, 마늘, 당근, 버섯을 추가하고 함께 볶아주세요.

5. **와인 넣기**: 모든 재료가 잘 섞이면 레드 와인을 부어주세요. 타임과 월계수 잎을 추가한 후 끓입니다.

6. **조리**: 끓어오르면 불을 줄이고 뚜껑을 덮은 채로 약 1시간 동안 낮은 온도에서 조리합니다. 중간에 한두 번 저어주세요.

7. **마무리**: 조리가 끝나면 베이컨을 다시 넣고 맛을 조절한 후, 파슬리로 장식하여 서빙합니다.

이 요리는 바게트와 함께 즐기면 더욱 좋습니다. 와인의 풍미가 잘 배어든 닭고기와 야채는 정말 환상적인 맛을 제공합니다. 즐거운 요리 되세요!


파서는 스트링이 아닌 json 형식으로도 만들 수 있습니다.   
프롬프트에서 형식을 요청하고, 이를 파서와 결합하여 변환하는 방식입니다.

In [16]:
from langchain_core.output_parsers import JsonOutputParser

jsonparser = JsonOutputParser()

In [17]:
jsonparser

JsonOutputParser()

In [18]:
jsonparser.get_format_instructions()

'Return a JSON object.'

In [19]:
recipe_template=ChatPromptTemplate.from_messages([
    ('system','당신은 전세계의 이색적인 퓨전 조리법의 전문가입니다.'),
    ('user','''저는 {ingredient}를 이용한 환상적인 파인 다이닝을 만들고 싶습니다. 추천해주세요!
    레시피에 대한 정보를 JSON 형식으로 출력해주세요.''')
])

recipe_chain = recipe_template | llm | jsonparser


In [20]:
response = recipe_chain.invoke({'ingredient':'콜라'})
response

{'recipe': {'name': '콜라 글레이즈드 스테이크',
  'cuisine': '퓨전',
  'servings': 4,
  'ingredients': [{'name': '소고기 스테이크',
    'quantity': '4 조각',
    'notes': '등심 또는 안심 추천'},
   {'name': '콜라', 'quantity': '1 컵', 'notes': '다이어트 콜라도 가능'},
   {'name': '간장', 'quantity': '1/4 컵', 'notes': '저염 간장 추천'},
   {'name': '마늘', 'quantity': '4 쪽', 'notes': '다진 마늘'},
   {'name': '생강', 'quantity': '1 작은술', 'notes': '다진 생강'},
   {'name': '올리브 오일', 'quantity': '2 큰술', 'notes': '스테이크 굽기용'},
   {'name': '소금', 'quantity': '적당량', 'notes': '취향에 따라 조절'},
   {'name': '후추', 'quantity': '적당량', 'notes': '취향에 따라 조절'},
   {'name': '파슬리', 'quantity': '약간', 'notes': '장식용'}],
  'instructions': ['1. 콜라, 간장, 다진 마늘, 다진 생강을 섞어 소스를 만든다.',
   '2. 스테이크에 소금과 후추로 간을 한다.',
   '3. 중불로 달군 팬에 올리브 오일을 두르고 스테이크를 양면이 갈색이 될 때까지 굽는다. (약 3-4분씩)',
   '4. 스테이크가 거의 다 구워지면 만든 콜라 소스를 팬에 붓고 중불에서 끓인다.',
   '5. 소스가 걸쭉해질 때까지 스테이크를 팬에 두고 조리한다. (약 2-3분)',
   '6. 스테이크를 접시에 담고 소스를 위에 뿌린 후, 파슬리로 장식한다.',
   '7. 뜨겁게 서빙한다.'],
  'cooking_time': {'prep_time': '10 분'

Json으로 파싱하는 방법은 활용도가 높지만, 실행할 때마다 결과뿐만 아니라 형식도 달라진다는 문제가 있습니다.

In [21]:
response = recipe_chain.invoke({'ingredient':'콜라'})
response

{'recipe': {'name': '콜라 글레이즈드 베이비 백립',
  'servings': 4,
  'ingredients': [{'name': '베이비 백립', 'quantity': '1 kg'},
   {'name': '콜라', 'quantity': '1 컵'},
   {'name': '간장', 'quantity': '1/4 컵'},
   {'name': '꿀', 'quantity': '2 큰술'},
   {'name': '마늘', 'quantity': '4 쪽 (다진 것)'},
   {'name': '생강', 'quantity': '1 작은술 (다진 것)'},
   {'name': '후추', 'quantity': '1 작은술'},
   {'name': '고춧가루', 'quantity': '1 작은술 (선택 사항)'},
   {'name': '소금', 'quantity': '적당량'},
   {'name': '올리브 오일', 'quantity': '2 큰술'},
   {'name': '다진 파슬리', 'quantity': '장식용'}],
  'instructions': ['오븐을 160도(섭씨)로 예열합니다.',
   '베이비 백립을 소금과 후추로 간을 합니다.',
   '팬에 올리브 오일을 두르고, 베이비 백립을 겉면이 갈색으로 변할 때까지 구워줍니다.',
   '콜라, 간장, 꿀, 다진 마늘, 다진 생강, 후추, 고춧가루를 혼합하여 소스를 만듭니다.',
   '구운 백립에 소스를 발라줍니다.',
   '예열된 오븐에 백립을 넣고 2시간 동안 조리합니다. 중간에 소스를 덧발라줍니다.',
   '조리가 끝나면 오븐에서 꺼내고, 다진 파슬리로 장식하여 서빙합니다.'],
  'cooking_time': {'prep_time': '20 분',
   'cook_time': '2 시간',
   'total_time': '2 시간 20 분'},
  'notes': '콜라의 단맛이 고기의 풍미를 살려줍니다. 매운 맛을 원하시면 고춧가루를 추가하세요.'}}

#### Pydantic을 이용해 확실한 형식 지정하기

pydantic은 데이터 형식에 제약조건을 두고 이를 준수하는지 검증하는 라이브러리입니다.


In [22]:
from pydantic import BaseModel, Field
# pydantic 연동

class Recipe(BaseModel):
    name: str = Field(description="음식 이름")
    # name: 문자열, 설명은 "음식 이름"
    difficulty: str = Field(description="만들기의 난이도")
    kick: str = Field(description='맛의 포인트')

    origin: str = Field(description="원산지")
    ingredients: list[str] = Field(description="재료")
    # ingredients: 문자열 리스트, 설명은 "재료"

    instructions: list[str] = Field(description="조리법")
    tip: str = Field(description='조리 과정 팁')


In [23]:
parser = JsonOutputParser(pydantic_object=Recipe)

In [24]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "음식 이름", "title": "Name", "type": "string"}, "difficulty": {"description": "만들기의 난이도", "title": "Difficulty", "type": "string"}, "kick": {"description": "맛의 포인트", "title": "Kick", "type": "string"}, "origin": {"description": "원산지", "title": "Origin", "type": "string"}, "ingredients": {"description": "재료", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "instructions": {"description": "조리법", "items": {"type": "string"}, "title": "Instructions", "type": "array"}, "tip": {"description

In [ ]:
# depreciated
# refined_json_instructions = parser.get_format_instructions().encode().decode('unicode_escape')
# print(refined_json_instructions)

In [25]:
recipe_template2 =ChatPromptTemplate.from_messages([
    ('system','당신은 전세계의 이색적인 퓨전 조리법의 전문가입니다.'),
    ('user','''저는 {ingredient}를 이용한 환상적인 외국 음식을 만들고 싶습니다. 추천해주세요!
    레시피에 대한 정보를 JSON 형식으로 출력해주세요. 결과는 한국어로 작성하세요.
     {instruction}''')
])

recipe_chain2 = recipe_template2 | llm | parser


In [26]:
recipe_template2

ChatPromptTemplate(input_variables=['ingredient', 'instruction'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 전세계의 이색적인 퓨전 조리법의 전문가입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ingredient', 'instruction'], input_types={}, partial_variables={}, template='저는 {ingredient}를 이용한 환상적인 외국 음식을 만들고 싶습니다. 추천해주세요!\n    레시피에 대한 정보를 JSON 형식으로 출력해주세요. 결과는 한국어로 작성하세요.\n     {instruction}'), additional_kwargs={})])

In [27]:
recipe_chain2.invoke({'ingredient':'마늘', 'instruction':parser.get_format_instructions}) #refined_json_instructions})

{'레시피명': '마늘 버터 새우',
 '재료': [{'이름': '새우', '수량': '500g'},
  {'이름': '버터', '수량': '100g'},
  {'이름': '마늘', '수량': '6쪽'},
  {'이름': '올리브유', '수량': '2큰술'},
  {'이름': '소금', '수량': '적당량'},
  {'이름': '후추', '수량': '적당량'},
  {'이름': '파슬리 (선택사항)', '수량': '다진 것 2큰술'}],
 '조리법': ['새우를 깨끗이 씻고 껍질을 벗긴 후, 소금과 후추로 간을 해둡니다.',
  '팬에 올리브유와 버터를 넣고 중불로 가열합니다.',
  '버터가 녹으면 다진 마늘을 넣고 향이 올라올 때까지 볶습니다.',
  '마늘 향이 올라오면 준비한 새우를 넣고, 새우가 핑크색으로 변할 때까지 볶습니다.',
  '새우가 다 익으면 불을 끄고, 원하면 다진 파슬리를 뿌립니다.',
  '뜨거운 상태로 서빙합니다.'],
 '서빙 제안': '밥이나 빵과 함께 곁들여 드시면 좋습니다.',
 '조리 시간': '15분',
 '난이도': '쉬움'}

#### Structured Output
LangChain의 Structured_Output 기능을 사용할 수도 있습니다.

In [28]:
structured_llm = llm.with_structured_output(Recipe)
structured_llm.invoke("생강으로 만들 수 있는 요리 레시피 알려주세요.")

Recipe(name='생강차', difficulty='쉬움', kick='상큼하고 따뜻한 맛', origin='한국', ingredients=['신선한 생강 100g', '물 4컵', '꿀 또는 설탕(기호에 따라)', '레몬 슬라이스 (선택 사항)'], instructions=['생강을 깨끗이 씻고 껍질을 벗긴 후 얇게 썬다.', '냄비에 물을 붓고 썬 생강을 넣는다.', '중불에서 물이 끓기 시작하면 불을 줄이고 10~15분 정도 끓인다.', '불을 끄고 기호에 따라 꿀이나 설탕을 넣어 단맛을 조절한다.', '컵에 담고 원한다면 레몬 슬라이스를 추가하여 즐긴다.'], tip='생강차는 감기 예방에 좋으며, 생강의 매운맛을 줄이기 위해 물에 오래 끓일수록 부드러운 맛이 난다.')

In [29]:
structured_llm = llm.with_structured_output(Recipe, method='json_mode')
structured_llm.invoke("생강으로 만들 수 있는 요리 레시피 알려주세요."+parser.get_format_instructions())

Recipe(name='생강차', difficulty='쉬움', kick='상큼하고 따뜻한 맛', origin='한국', ingredients=['생강 100g', '물 500ml', '꿀 또는 설탕 적당량', '레몬 슬라이스 (선택사항)'], instructions=['생강을 깨끗이 씻고 껍질을 벗긴 후 얇게 썬다.', '냄비에 물을 붓고 생강을 넣고 끓인다.', '물이 끓기 시작하면 중불로 줄이고 10-15분간 더 끓인다.', '불을 끄고 체에 걸러 생강을 제거한다.', '원하는 만큼 꿀 또는 설탕을 넣고 잘 저어준다.', '레몬 슬라이스를 추가하여 맛을 더할 수 있다.'], tip='생강차는 따뜻하게 마시는 것이 좋으며, 몸이 차가울 때 마시면 효과적입니다.')

이제, 댓글이 주어지면 댓글의 답변을 생성해 보겠습니다.

In [30]:
import pandas as pd
reviews = pd.read_csv('./1_1.data_reviews.csv')
print(reviews.shape)
reviews.head()

(50, 3)


,Num,Review,Label
0,1,그닥 맛있고 좋은 고기인지는 모르겠내요 테이블 나누어서 두팀 받는데 옆 테이블 시끄...,-1
1,2,적당히 좋은 소고기를 싸지 않은 가격에 맛볼 수 있다. 하지만 인기나 리뷰에 비해선...,-1
2,3,기름진맛 역시맛있네요 많이먹긴 힘들지만 맛있는 소고기집인건 틀림없어요,1
3,4,"지인이 추천하고 짬뽕맛집이래서 찾아갔는데, 주차도 골목길에 해야되고 맛도 별로네요",-1
4,5,매운단계별짬뽕과 불향가득짜장 직접만든소스가별미인탕수육 제입맛에는딱맞아서자주찾게되요^^*,1


In [31]:
reply_template = ChatPromptTemplate.from_messages([
    ('system','''당신은 레스토랑의 주인입니다.
고객이 다음과 같은 리뷰를 남겼을 때, 답변을 작성해 주세요.
첫 문장은 가상의 레스토랑 이름과 함께 인사하는 내용을 포함하세요.
고객의 의견에 매우 공감하여 답변하고, 부정적인 피드백은 사과하세요.
새로운 메뉴나 프로모션을 홍보호가, 재방문을 기원하세요.
밝고 유쾌한 톤으로 작성하고, 이모지를 매우 많이 추가하세요.

'''),
    ('user','''{review}''')

])

In [32]:
reply_chain = reply_template | llm | StrOutputParser()
reply_chain.invoke({'review':reviews['Review'][0]})

'안녕하세요! 🌟 저희 레스토랑 "맛있는 순간"에 방문해 주셔서 정말 감사드립니다! 😊\n\n고객님께서 경험하신 불편함에 대해 진심으로 사과드립니다. 😔 저희는 항상 고객님들께 최고의 식사 경험을 제공하고자 노력하고 있지만, 이번에는 기대에 미치지 못한 것 같아 매우 아쉽습니다. 고기 맛과 소음 문제에 대해 더욱 신경 쓰도록 하겠습니다! 🙏\n\n다행히도 저희는 새로운 메뉴를 출시했답니다! 🍖✨ 이번 주부터 특별한 양념 소고기와 함께 다양한 사이드 메뉴를 즐기실 수 있어요! 🎉 또한, 다음 방문 시 사용할 수 있는 할인 쿠폰도 준비해두었으니 꼭 활용해 보세요! 💌\n\n다시 한 번 저희 레스토랑을 찾아주신다면 더 좋은 경험을 선사해드릴 수 있도록 최선을 다하겠습니다! 😊💕 언제든지 환영합니다! 🌈✨'

입력이 여러 개인 경우, 이를 리스트로 만들고 batch()를 이용해 한꺼번에 입력할 수 있습니다.

In [33]:
reviews.loc[:5,'Review'].to_list()

['그닥 맛있고 좋은 고기인지는 모르겠내요 테이블 나누어서 두팀 받는데 옆 테이블 시끄러워서 짜증이 많이 나내요',
 '적당히 좋은 소고기를 싸지 않은 가격에 맛볼 수 있다. 하지만 인기나 리뷰에 비해선 평범한 수준. 이런 맛과 가성비의 소고기집은 동네마다 아주아주 많다.',
 '기름진맛 역시맛있네요 많이먹긴 힘들지만 맛있는 소고기집인건 틀림없어요',
 '지인이 추천하고 짬뽕맛집이래서 찾아갔는데, 주차도 골목길에 해야되고 맛도 별로네요',
 '매운단계별짬뽕과 불향가득짜장 직접만든소스가별미인탕수육 제입맛에는딱맞아서자주찾게되요^^*',
 '신맛나지 않는 달콤한 탕수육도 맛있다 친절한 사장님도 추가 당분간 맛없는 배달짬뽕 먹을 생각하니 슬프다 사장님 쾌차하세요']

In [34]:
reply_chain.batch(reviews.loc[:5,'Review'].to_list())

['안녕하세요! 🎉 저희 레스토랑 "맛있는 고기집"에 방문해 주셔서 정말 감사드립니다! 😊 고객님께서 말씀하신 부분에 깊이 공감하며, 불편을 드려서 진심으로 사과드립니다. 😔 저희는 항상 고객님들께 최고의 경험을 제공하기 위해 노력하고 있지만, 이번에는 기대에 미치지 못한 것 같아 아쉽습니다. \n\n저희는 새로운 메뉴를 준비 중이며, 이번 주에는 특별 프로모션도 진행하고 있어요! 🍽️🥳 다음 번 방문 시에는 더욱 맛있는 고기와 함께 편안한 분위기를 즐기실 수 있도록 최선을 다하겠습니다. \n\n다시 한 번 저희 레스토랑에 와주신 것에 감사드리며, 다음 방문을 손꼽아 기다리겠습니다! 💖😊 좋은 하루 되세요! 🌟',
 '안녕하세요! 🍽️ "맛있는 소고기 집"에 오신 것을 환영합니다! 😊\n\n소중한 리뷰 남겨주셔서 정말 감사합니다! 🙏 고객님께서 말씀하신 것처럼, 저희가 더 특별한 경험을 제공해 드리지 못한 점 정말 죄송합니다. 😔 저희는 항상 고객님의 기대에 부응하기 위해 노력하고 있으며, 앞으로 더 나은 맛과 서비스를 제공할 수 있도록 최선을 다하겠습니다! 💪✨\n\n최근 저희는 새로운 메뉴를 출시했답니다! 🍖😍 특히 소고기를 더욱 맛있게 즐길 수 있는 특별한 소스와 함께 제공하고 있으니, 다음 방문 때 꼭 한번 시도해 보세요! 🎉 또한, 현재 재방문 고객님들을 위한 특별 프로모션도 진행 중이니 놓치지 마세요! 🎈\n\n다시 한번 저희 레스토랑을 찾아주시면 더 나은 경험을 드릴 수 있도록 준비하겠습니다! 🌟 감사합니다! 행복한 하루 되세요! 💖🍀',
 '안녕하세요! 🍽️ 환상적인 맛을 자랑하는 "소고기 천국"에 오신 것을 환영합니다! 😊 고객님께서 기름진 맛이 맛있다고 해주셔서 정말 기쁩니다! 🎉 하지만 많이 드시기 힘드셨다니, 그 점에 대해서는 사과드립니다. 😔 저희는 항상 고객님의 건강과 편안함을 최우선으로 생각하고 있답니다! 💖\n\n저희는 이번 주부터 새로운 메뉴를 출시했어요! 🍖✨ 특별한 소스와 함께 즐길 수 있는 소고기 구이가 준비되어 있으니, 꼭

프롬프트를 잘 구성하거나, Schema를 사용한다면 답글의 형식을 통일할 수 있습니다.

#### Runnables

Runnables는 LCEL의 기본 단위로, 입력을 받아 출력을 생성하는 기본 단위입니다.    
llm, prompt, chain 등이 모두 Runnable 구조에 해당합니다.

이번에는, 데이터 흐름을 제어하는 특별한 Runnable인   
RunnablePassthrough와 RunnableParallel을 이용해 체인을 구성해 보겠습니다.



#### RunnablePassthrough
RunnablePassthrough는 체인의 직전 출력을 그대로 가져옵니다.

In [35]:
from langchain.schema.runnable import RunnablePassthrough

prompt1 = ChatPromptTemplate.from_template(
    "{director}의 대표 작품은 무엇입니까?")
chain1 = (
    prompt1
    | llm
    | StrOutputParser()
    | {'answer': RunnablePassthrough()})

response = chain1.invoke("스티븐 스필버그")
print(response)

{'answer': "스티븐 스필버그는 많은 유명한 영화들을 감독한 할리우드의 전설적인 영화 감독입니다. 그의 대표 작품으로는 다음과 같은 영화들이 있습니다:\n\n1. **죠스 (Jaws, 1975)** - 현대적인 블록버스터의 시작을 알린 영화로, 상어의 공포를 다룹니다.\n2. **E.T. (E.T. the Extra-Terrestrial, 1982)** - 외계인과 소년의 우정을 그린 감동적인 이야기입니다.\n3. **인디아나 존스 시리즈 (Indiana Jones Series)** - 모험과 액션을 결합한 이 시리즈는 전 세계적으로 큰 인기를 끌었습니다.\n4. **쥬라기 공원 (Jurassic Park, 1993)** - 공룡을 복원한 테마파크의 이야기를 다룬 혁신적인 특수 효과의 영화입니다.\n5. **쉰들러 리스트 (Schindler's List, 1993)** - 홀로코스트를 배경으로 한 감동적인 실화를 다룬 작품으로, 아카데미상을 수상했습니다.\n6. **라이언 일병 구하기 (Saving Private Ryan, 1998)** - 제2차 세계대전을 배경으로 한 전쟁 영화로, 사실적인 전투 장면이 인상적입니다.\n7. **미지의 세계 (Minority Report, 2002)** - 미래를 예측하는 기술을 다룬 사이버펑크 영화입니다.\n8. **링컨 (Lincoln, 2012)** - 아브라함 링컨 대통령의 생애와 그의 정치적 투쟁을 다룬 작품입니다.\n\n이 외에도 스필버그는 수많은 다른 작품들을 감독하며 영화 역사에 큰 영향을 미쳤습니다."}


<br><br>
### RunnableParallel

RunnableParallel은 서로 다른 체인을 병렬적으로 실행합니다.

In [36]:
from langchain_core.runnables import RunnableParallel

prompt1 = ChatPromptTemplate.from_template( # 괄호 뒤에서 줄바꿈하면 다음 줄도 같은 줄로 인식
    "색깔을 하나 알려주세요, 색깔만 출력하세요.")
prompt2 = ChatPromptTemplate.from_template(
    "음식을 하나 알려주세요, 음식만 출력하세요.")

chain1 = prompt1 | llm | StrOutputParser()
chain2 = prompt2 | llm | StrOutputParser()

chain3 = RunnableParallel(color = chain1, food = chain2)

chain3.invoke({}) # chain3.invoke()는 에러


{'color': '파랑', 'food': '김치찌개'}

<br><br><br>이번에는 LLM의 결과를 다음 LLM으로 연결해 보겠습니다.

In [37]:
prompt1 = ChatPromptTemplate.from_template("잭슨빌은 어느 나라의 도시입니까? 나라이름만 출력하세요.")
prompt2 = ChatPromptTemplate.from_template(
    "{country}의 대표적인 인물 3명을 나열하세요. 인물의 이름만 출력하세요."
)

chain1 = prompt1 | llm | StrOutputParser()
chain2 =(
    {"country": chain1} | prompt2 | llm | StrOutputParser()
)
chain2.invoke({})

'조지 워싱턴, 에이브러햄 링컨, 마틴 루터 킹 주니어'

RunnableParallel을 사용하면 chain1의 결과와 chain2의 결과를 함께 얻을 수 있습니다.   
이 때, chain1의 실행 결과를 chain2에 전달하는 방식으로 실행됩니다.

In [38]:
chain1 = prompt1 | llm | StrOutputParser()
chain2 = prompt2 | llm | StrOutputParser()  # chain2 형식이 바뀜

chain3 = RunnableParallel(country = chain1).assign(people = chain2)

chain3.invoke({})

{'country': '미국', 'people': '조지 워싱턴, 에이브러햄 링컨, 마틴 루터 킹 주니어'}

#### 실습) 체인 연결하기

위 코드들을 참고하여, LLM을 3개 연결해서 3개 결과를 출력해 보세요.   
체인 3개로 만들 수도 있고, RunnableParallel을 사용할 수도 있습니다.   

예시)    
1. 영화->감독
2. 감독->가장 유명한 배우
3. 배우->상을 받은 영화

In [39]:
# 예시 프롬프트
prompt1 = ChatPromptTemplate.from_template("{movie}의 감독은 누구입니까? 감독 이름만 출력하세요.")
prompt2 = ChatPromptTemplate.from_template("{director}와 작업한 가장 유명한 배우는 누구인가요? 배우 이름만 출력하세요.")
prompt3 = ChatPromptTemplate.from_template("{actor}는 무슨 영화로 상을 받았나요?")



In [40]:
chain1 = prompt1 | llm | StrOutputParser()
chain2 = prompt2 | llm | StrOutputParser()
chain3 = prompt3 | llm | StrOutputParser()

chain4 = RunnableParallel(director = chain1).assign(actor = chain2).assign(award = chain3)

chain4.invoke('벤허')


{'director': '윌리엄 와일러',
 'actor': '오드리 헵번',
 'award': '오드리 헵번은 1954년 영화 "로마의 휴일"로 아카데미 시상식에서 여우주연상을 수상했습니다. 이 영화에서 그녀는 공주 역할을 맡아 많은 사랑을 받았으며, 이 작품은 그녀의 대표작 중 하나로 여겨집니다.'}

<br><br>
chain2에서 새로운 매개변수가 추가되는 경우는 어떻게 해야 할까요?

Lambda 함수를 통해, 입력 dict로부터 값을 선택합니다.

In [41]:
prompt1 = ChatPromptTemplate.from_template("{city}는 어느 나라의 도시인가요? 나라 이름만 출력하세요.")
prompt2 = ChatPromptTemplate.from_template(
    "{country}의 유명한 인물은 누가 있나요? {num} 명의 이름을 나열하세요. 사람 이름만 ,로 구분하여 나열하세요."
)

chain1 = prompt1 | llm | StrOutputParser()

chain2 = (
    RunnableParallel(country = chain1, num = lambda x:x['num']) # lambda x:f(x) --> x를 입력 받으면 f(x)를 return, x는 직전에 입력된 값
    # RunnableParallel은 여러 개의 체인을 병렬로 실행하는 기능을 하며, 여기서 lambda x:x["num"]는 입력 데이터에서 num 값을 그대로 가져오도록 설정
    # 그럼 여기서 lambda를 사용하지 않고 num = chain1을 하면? RunnableParallel(country=chain1, num=chain1)에서 chain1은 country 값을 반환하지만, num도 chain1에 전달되면서 LLM이 num을 국가처럼 해석할 수 있음. 즉, "잭슨빌" → "미국" 변환을 num에도 적용하려고 시도하면서 잘못된 결과가 나올 가능성이 높음
    # invoke에서 주어지는 dict를 전처리한 것
    | prompt2
    | llm
    | StrOutputParser()
)

print(chain2.invoke({"city": "잭슨빌", "num": "3"}))

조지 워싱턴, 마틴 루터 킹 주니어, 앨버트 아인슈타인


<br><br>
체인을 분리하고 RunnableParallel을 이용하면 중간 과정을 모두 출력할 수 있습니다.

In [42]:
chain4 = (prompt2
    | llm
    | StrOutputParser())

chain3 = RunnableParallel(country = chain1, num = lambda x:x['num']).assign(res = chain4)

chain3.invoke({"city": "코펜하겐", "num": "3"})

{'country': '덴마크', 'num': '3', 'res': '한스 크리스티안 안데르센, 소렌 키르케고르, 마르그레테 2세'}

<br><br><br>JsonOutputParser를 쓴다면 아래와 같이 만들 수도 있습니다.

In [43]:
from langchain_core.output_parsers import JsonOutputParser

prompt1 = ChatPromptTemplate.from_template("미국 영화 배우와 대표작을 하나 나열하세요. json 형식으로 출력하고, 각 항목은 actor, movie로 표시하세요.")
prompt2 = ChatPromptTemplate.from_template(
    "{actor}는 {movie}에서 어떤 역할을 했습니까?"
)

chain1 = prompt1 | llm | JsonOutputParser()
chain2 =(
     chain1 | prompt2 | llm | StrOutputParser()
)
chain2.invoke({})

'Leonardo DiCaprio는 영화 "Inception"에서 주인공인 돔 코브(Dom Cobb) 역할을 맡았습니다. 그는 꿈을 조작하고 타인의 잠재의식을 침투하는 전문 도둑으로, 복잡한 꿈의 세계에서 임무를 수행하는 과정을 그립니다. 코브는 자신의 과거와 가족에 대한 갈등을 안고 있으며, 영화의 주요 플롯은 그가 자신의 죄책감과 싸우면서 새로운 삶을 찾으려는 여정을 중심으로 전개됩니다.'